In [10]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config(path="Users/odl_user_244707/config.json")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

StatementMeta(536407ba-818a-40ee-88f3-c0aa59ce8765, 0, 33, Finished, Available)

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code BBR5BG8E4 to authenticate.
Interactive authentication successfully completed.
Workspace name: quick-starts-ws-244707
Azure region: southcentralus
Subscription id: 9b72f9e6-56c5-4c16-991b-19c652994860
Resource group: aml-quickstarts-244707


In [11]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "projectrubric"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
try:
    compute_target = ComputeTarget(workspace=ws,name=cluster_name)
    print("found existing cluster use it")
except ComputeTargetException:
    print("creating a new compute")
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2",min_nodes=1,max_nodes=4)
    compute_target = ComputeTarget.create(ws,cluster_name,compute_config)

compute_target.wait_for_completion(show_output=True)
print(compute_target.get_status().serialize())



StatementMeta(536407ba-818a-40ee-88f3-c0aa59ce8765, 0, 34, Finished, Available)

found existing cluster use it
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 1, 'targetNodeCount': 1, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 1, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2023-12-03T15:12:07.219000+00:00', 'errors': None, 'creationTime': '2023-12-03T15:10:47.191349+00:00', 'modifiedTime': '2023-12-03T15:10:56.924652+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 1, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT1800S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


In [5]:
%pip install azureml-sdk 

StatementMeta(, , -1, Finished, Available)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.7/137.7 KB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 63.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 41.0 MB/s eta 0:00:0000:01:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 656.6/656.6 KB 64.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 KB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.8/224.8 KB 39.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.7/97.7 KB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 72.1 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 66.8 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 76.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 KB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.6/104.6 KB 26.9 MB/s eta 0:00:00
     

In [17]:
#from azureml.widgets import RunDetails
#from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
### YOUR CODE HERE ###
ps = RandomParameterSampling(
    {
        '--C':choice(0.001,0.01,1,10,20,50,100,200,500,1000),
        '--max_iter':choice(50,100,200,300)
    }
)

# Specify a Policy
 ### YOUR CODE HERE ###
policy = BanditPolicy(evaluation_interval=2,slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='Users/odl_user_244707/conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
 ### YOUR CODE HERE ###
src = ScriptRunConfig(source_directory='.',script='train.py',compute_target=compute_target,environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
 ### YOUR CODE HERE ###

hyperdrive_config = HyperDriveConfig(run_config=src,hyperparameter_sampling=ps,policy=policy,primary_metric_name='accuracy',primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,max_total_runs=20,max_concurrent_runs=4) 

StatementMeta(536407ba-818a-40ee-88f3-c0aa59ce8765, 0, 40, Finished, Available)

In [8]:
%pip install azureml-widgets

StatementMeta(, , -1, Finished, Available)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 60.6 MB/s eta 0:00:0000:0100:01
You should consider upgrading via the '/nfs4/pyenv-575d8c99-64c0-4a38-be47-ad157d163db2/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [18]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
from azureml.widgets import RunDetails
### YOUR CODE HERE ###
run_hyp = exp.submit(hyperdrive_config)
RunDetails(run_hyp).show()
run_hyp.wait_for_completion(show_output=True)
assert(hyperdrive_config.get_status()=="Completed")

StatementMeta(536407ba-818a-40ee-88f3-c0aa59ce8765, 0, 41, Submitted, Running)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_8bc238c2-6185-4012-aec8-87903ad2bae9
Web View: https://ml.azure.com/runs/HD_8bc238c2-6185-4012-aec8-87903ad2bae9?wsid=/subscriptions/9b72f9e6-56c5-4c16-991b-19c652994860/resourcegroups/aml-quickstarts-244707/workspaces/quick-starts-ws-244707&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2023-12-03T15:29:53.112315][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2023-12-03T15:29:53.5172522Z][SCHEDULER][INFO]Scheduling job, id='HD_8bc238c2-6185-4012-aec8-87903ad2bae9_0' 
[2023-12-03T15:29:53.6257924Z][SCHEDULER][INFO]Scheduling job, id='HD_8bc238c2-6185-4012-aec8-87903ad2bae9_1' 
[2023-12-03T15:29:53.7842778Z][SCHEDULER][INFO]Scheduling job, id='HD_8bc238c2-6185-4012-aec8-87903ad2bae9_2' 
[2023-12-03T15:29:53.9167084Z][SCHEDULER][INFO]Scheduling job, id='HD_8bc238c2-6185-4012-aec8-87903ad2bae9_3' 
[2023-12-03T15:29:53.863533][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution t

In [ ]:
import joblib

In [ ]:
import joblib
# Get your best run and save the model from that run.
print(hyperdrive_config.get_children_sorted_by_primary_metric(top=0,reverse=False,discard_no_metric=False))
best_run = hyperdrive_config.get_best_run_by_primary_metric()
#get_metrics()
#Returns the metric from all the runs that were launched by this HyperdriveRun.
print("Best run metrics " ,best_run.get_metrics())
### YOUR CODE HERE ###
#get_details()
#Returns a dictionary with the details for the run
print("Best run details :",best_run.get_details())
#get_file_names()
#Returns a list of the files that are stored in association with the run.
print("Best run file names :",best_run.get_file_names())

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
ds = TabularDatasetFactory.from_delimited_files(['https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'])

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=ds,
    label_column_name=y,
    n_cross_validations=2)

In [2]:
# Submit your automl run

### YOUR CODE HERE ###
remote_run = exp.submit(automl_config,show_output=False)
remote_run.wait_for_completion()

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_run,fitted_model = remote_run.get_output()
print(best_run)
print(fitted_model)

#get_metrics
#Returns the metrics
print("Best run metrics :",best_run.get_metrics())
#get_details()
#Returns a dictionary with details for the run
print("Best run details : ",best_run.get_details())

In [ ]:
best_run.get_metrics()

In [ ]:
fitted_model

In [ ]:
best_run

In [ ]:
compute_target.delete()